# Quantum Error Correction Visualization Tool

This notebook provides an interactive demonstration of quantum error correction codes.

## Features:
- 3-qubit bit-flip code
- 5-qubit perfect code
- Multiple error types (bit-flip, phase-flip, depolarizing, Rx gate)
- Interactive visualizations
- Step-by-step QEC process


In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt

# Add parent directory to path
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

from qec_visualizer import (
    BitFlipCode, PerfectCode, ErrorInjector, ErrorType,
    QECVisualizer, QECBackend
)

print("Imports successful!")


## Interactive QEC Demo

Use the widgets below to interact with the QEC visualization tool.


In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

def interactive_qec_demo(
    code_type='Bit-Flip Code',
    logical_state=0,
    error_type='Bit-Flip',
    error_qubit=0,
    error_probability=1.0,
    rotation_angle=0.785
):
    """Interactive QEC demonstration."""
    
    # Select code
    if code_type == 'Bit-Flip Code':
        code = BitFlipCode()
        n_qubits = 3
    else:
        code = PerfectCode()
        n_qubits = 5
    
    # Map error type string to enum
    error_type_map = {
        'Bit-Flip': ErrorType.BIT_FLIP,
        'Phase-Flip': ErrorType.PHASE_FLIP,
        'Depolarizing': ErrorType.DEPOLARIZING,
        'Rx Gate': ErrorType.ROTATION_X,
        'Ry Gate': ErrorType.ROTATION_Y,
        'Rz Gate': ErrorType.ROTATION_Z
    }
    
    error_enum = error_type_map[error_type]
    
    # Initialize components
    error_injector = ErrorInjector(n_qubits=n_qubits)
    visualizer = QECVisualizer()
    backend = QECBackend()
    
    # Step 1: Encode
    encoding_circuit = code.encode(logical_state=logical_state)
    
    # Step 2: Inject error
    if error_enum in [ErrorType.ROTATION_X, ErrorType.ROTATION_Y, ErrorType.ROTATION_Z]:
        error_circuit = error_injector.inject_error(
            encoding_circuit,
            error_enum,
            qubit=error_qubit,
            error_probability=error_probability,
            rotation_angle=rotation_angle
        )
    else:
        error_circuit = error_injector.inject_error(
            encoding_circuit,
            error_enum,
            qubit=error_qubit,
            error_probability=error_probability
        )
    
    # Step 3: Measure syndrome
    syndrome_circuit = code.syndrome_measurement()
    syndrome = backend.extract_syndrome(error_circuit, syndrome_circuit)
    
    # Step 4: Correct
    correction_circuit = code.correct(syndrome)
    
    # Step 5: Decode
    decoding_circuit = code.decode()
    
    # Visualize
    print(f"Code: {code.name}")
    print(f"Logical State: |{logical_state}⟩")
    print(f"Error Type: {error_type}")
    print(f"Error Location: Qubit {error_qubit}")
    print(f"Syndrome: {syndrome}")
    
    # Get states
    initial_state = backend.get_statevector(encoding_circuit)
    error_state = backend.get_statevector(error_circuit)
    
    # Calculate fidelity
    fidelity = backend.calculate_fidelity(initial_state, error_state)
    print(f"Fidelity after error: {fidelity:.4f}")
    
    # Plot comparison
    visualizer.plot_probability_comparison(
        [initial_state, error_state],
        ['Initial State', 'After Error'],
        title=f"QEC Process - {code.name}"
    )
    
    # Plot circuit
    visualizer.plot_circuit_diagram(encoding_circuit, title="Encoding Circuit")

# Create interactive widget
interact(
    interactive_qec_demo,
    code_type=widgets.Dropdown(
        options=['Bit-Flip Code', 'Perfect Code'],
        value='Bit-Flip Code',
        description='QEC Code:'
    ),
    logical_state=widgets.IntSlider(
        min=0, max=1, value=0, description='Logical State:'
    ),
    error_type=widgets.Dropdown(
        options=['Bit-Flip', 'Phase-Flip', 'Depolarizing', 'Rx Gate', 'Ry Gate', 'Rz Gate'],
        value='Bit-Flip',
        description='Error Type:'
    ),
    error_qubit=widgets.IntSlider(
        min=0, max=4, value=0, description='Error Qubit:'
    ),
    error_probability=widgets.FloatSlider(
        min=0.0, max=1.0, value=1.0, step=0.1, description='Error Prob:'
    ),
    rotation_angle=widgets.FloatSlider(
        min=0.0, max=np.pi, value=np.pi/4, step=0.1, description='Rotation Angle:'
    )
)


## Step-by-Step Visualization

Visualize the complete QEC process step by step.


In [ ]:
def step_by_step_demo():
    """Demonstrate step-by-step QEC process."""
    
    code = BitFlipCode()
    error_injector = ErrorInjector(n_qubits=3)
    visualizer = QECVisualizer()
    backend = QECBackend()
    
    # Create all stages
    encoding_circuit = code.encode(logical_state=1)
    error_circuit = error_injector.inject_error(
        encoding_circuit, ErrorType.BIT_FLIP, qubit=0, error_probability=1.0
    )
    syndrome_circuit = code.syndrome_measurement()
    syndrome = backend.extract_syndrome(error_circuit, syndrome_circuit)
    correction_circuit = code.correct(syndrome)
    corrected_circuit = error_circuit.compose(correction_circuit)
    
    # Get states
    states = [
        backend.get_statevector(encoding_circuit),
        backend.get_statevector(error_circuit),
        backend.get_statevector(corrected_circuit)
    ]
    
    circuits = [encoding_circuit, error_circuit, corrected_circuit]
    stage_names = ['Encoding', 'After Error', 'After Correction']
    
    visualizer.create_step_by_step_visualization(
        circuits, states, stage_names,
        title="3-qubit Bit-Flip Code - Step by Step"
    )

step_by_step_demo()


## Fidelity Analysis

Analyze how different errors affect fidelity.


In [ ]:
def fidelity_analysis():
    """Analyze fidelity for different error types."""
    
    code = BitFlipCode()
    error_injector = ErrorInjector(n_qubits=3)
    visualizer = QECVisualizer()
    backend = QECBackend()
    
    encoding_circuit = code.encode(logical_state=0)
    initial_state = backend.get_statevector(encoding_circuit)
    
    error_types = [
        ErrorType.BIT_FLIP,
        ErrorType.PHASE_FLIP,
        ErrorType.ROTATION_X,
        ErrorType.DEPOLARIZING
    ]
    
    fidelities = []
    labels = []
    
    for error_type in error_types:
        error_circuit = error_injector.inject_error(
            encoding_circuit, error_type, qubit=0, error_probability=1.0
        )
        error_state = backend.get_statevector(error_circuit)
        fidelity = backend.calculate_fidelity(initial_state, error_state)
        
        fidelities.append(fidelity)
        labels.append(error_type.value.replace('_', ' ').title())
    
    visualizer.plot_fidelity_evolution(
        fidelities, labels,
        title="Fidelity Analysis - Different Error Types"
    )

fidelity_analysis()
